# Text Mining

In [3]:
import spacy
import pandas as pd
from collections import Counter
import gensim
from gensim import corpora, models
from gensim.corpora import Dictionary
from gensim.models import LdaModel
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import warnings
warnings.filterwarnings('ignore')

## Modelos del lenguaje

In [5]:
# !python -m spacy download es_core_news_sm
# Cargando el Language Model de español
nlp = spacy.load('es_core_news_sm')

In [6]:
# Procesando una oracion a traves del pipeline
doc = nlp('Esto es una oracion.')

## Part-of-Speech Tagging

In [7]:
doc = nlp('Juan y yo fuimos al parque')

In [8]:
for token in doc:
    print((token.text, token.pos_))

('Juan', 'PROPN')
('y', 'CCONJ')
('yo', 'PRON')
('fuimos', 'VERB')
('al', 'ADP')
('parque', 'NOUN')


## Named Entity Recognition Tagging

In [9]:
doc = nlp('Microsoft tiene oficinas por todo Europa.')

In [10]:
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Microsoft 0 9 ORG
Europa 34 40 LOC


## Ejemplo de preprocesamiento

In [11]:
doc = nlp('El caballo galopó por el campo y pasó los 2 ríos.')

In [12]:
sentence = []
for w in doc:
    # Si no es una stop word o signo de puntuacion, agregarlo al articulo
    if w.text != 'n' and not w.is_stop and not w.is_punct and not w.like_num:
        # Sumamos la version lemmatizada de la palabra
        sentence.append(w.lemma_)

In [13]:
print(sentence)

['caballo', 'galopó', 'campo', 'y', 'pasar', 'río']


---

## PoS Tagging y aplicaciones

In [14]:
sent_0 = nlp('Mateo y yo fuimos al parque.')
for token in sent_0:
    print(token.text, token.pos_, token.tag_)

Mateo PROPN PROPN
y CCONJ CCONJ
yo PRON PRON
fuimos VERB VERB
al ADP ADP
parque NOUN NOUN
. PUNCT PUNCT


Identificó a `Mateo` como **Sustantivo Propio** y a `parque` como **Sustantivo Comun**

In [15]:
sent_1 = nlp('Si a Clemente le piden que saque la basura, el se va a negar.')
for token in sent_1:
    print(token.text, token.pos_, token.tag_)

Si SCONJ SCONJ
a ADP ADP
Clemente PROPN PROPN
le PRON PRON
piden VERB VERB
que SCONJ SCONJ
saque VERB VERB
la DET DET
basura NOUN NOUN
, PUNCT PUNCT
el DET DET
se PRON PRON
va AUX AUX
a ADP ADP
negar VERB VERB
. PUNCT PUNCT


In [16]:
sent_2 = nlp('Bautista estuvo a cargo del centro de tratamiento de desechos.')
for token in sent_2:
    print(token.text, token.pos_, token.tag_)

Bautista PROPN PROPN
estuvo AUX AUX
a ADP ADP
cargo NOUN NOUN
del ADP ADP
centro NOUN NOUN
de ADP ADP
tratamiento NOUN NOUN
de ADP ADP
desechos NOUN NOUN
. PUNCT PUNCT


In [17]:
sent_3 = nlp('María pescó a su gato sospechoso y lo sacó a pescar pescado')
for token in sent_3:
    print(token.text, token.pos_, token.tag_)

María PROPN PROPN
pescó VERB VERB
a ADP ADP
su DET DET
gato NOUN NOUN
sospechoso ADJ ADJ
y CCONJ CCONJ
lo PRON PRON
sacó VERB VERB
a ADP ADP
pescar VERB VERB
pescado ADJ ADJ


`pescado` fue interpretado como adjetivo cuando deberia ser un sustantivo

## Ejemplo de uso de PoS

In [18]:
martin_fierro = open('corpus/martin-fierro.txt').read()
mf = nlp(martin_fierro)

In [19]:
mfSents = list(mf.sents)
mfSentenceLengths = [len(sent) for sent in mfSents]
[sent for sent in mfSents if len(sent) == max(mfSentenceLengths)]
mfPOS = pd.Series(mf.count_by(spacy.attrs.POS))/len(mf)

In [20]:
tagDict = {w.pos: w.pos_ for w in mf}
mfPOS = pd.Series(mf.count_by(spacy.attrs.POS))/len(mf)
df = pd.DataFrame([mfPOS], index=['Martin Fierro'])
df.columns = [tagDict[column] for column in df.columns]
df

,DET,PROPN,SPACE,ADP,PUNCT,NUM,NOUN,ADJ,PRON,AUX,VERB,SCONJ,CCONJ,ADV,INTJ
Martin Fierro,0.097135,0.015906,0.163333,0.085439,0.105497,0.025439,0.142632,0.030526,0.078538,0.026784,0.11655,0.035848,0.036784,0.038538,0.001053


In [21]:
mfAdjs = [w for w in mf if w.pos_ == 'NOUN']
Counter([w.text.strip() for w in mfAdjs]).most_common(20)

[('gaucho', 37),
 ('hombre', 24),
 ('día', 23),
 ('vez', 23),
 ('mesmo', 18),
 ('suerte', 18),
 ('vida', 17),
 ('ocasión', 16),
 ('gente', 16),
 ('amigo', 15),
 ('mujer', 15),
 ('fin', 14),
 ('cosa', 13),
 ('tierra', 12),
 ('ahi', 12),
 ('hijos', 11),
 ('punto', 11),
 ('noche', 11),
 ('corazón', 10),
 ('campo', 10)]

In [22]:
mfAdjs = [w for w in mf if w.pos_ == 'PROPN']
Counter([w.text.strip() for w in mfAdjs]).most_common(20)

[('Dios', 18),
 ('mas', 16),
 ('Fierro', 8),
 ('Cruz', 8),
 ('Ansí', 7),
 ('Mas', 4),
 ('Cantando', 4),
 ('hacienda', 4),
 ('usté', 4),
 ('Ahi', 4),
 ('jue', 4),
 ('Gaucho', 3),
 ('Martín', 3),
 ('MARTÍN', 3),
 ('FIERRO', 3),
 ('palo', 3),
 ('José', 2),
 ('Buenos', 2),
 ('Aires', 2),
 ('Santos', 2)]

In [23]:
mfAdjs = [w for w in mf if w.pos_ == 'VERB']
Counter([w.text.strip() for w in mfAdjs]).most_common(20)

[('tiene', 19),
 ('dende', 18),
 ('dio', 17),
 ('tenía', 16),
 ('sé', 15),
 ('ande', 14),
 ('ansí', 14),
 ('dije', 13),
 ('cantar', 12),
 ('andaba', 12),
 ('visto', 11),
 ('ver', 11),
 ('vino', 11),
 ('andar', 10),
 ('dijo', 10),
 ('tener', 10),
 ('hacer', 9),
 ('tengo', 9),
 ('hizo', 9),
 ('hice', 9)]

In [24]:
mfAdjs = [w for w in mf if w.pos_ == 'ADJ']
Counter([w.text.strip() for w in mfAdjs]).most_common(20)

[('gaucho', 18),
 ('pobre', 15),
 ('juntos', 7),
 ('mejor', 6),
 ('naides', 6),
 ('malo', 6),
 ('triste', 6),
 ('mayor', 5),
 ('muerto', 5),
 ('desgraciao', 5),
 ('amigo', 4),
 ('barajo', 4),
 ('solo', 4),
 ('siguro', 4),
 ('primero', 4),
 ('llena', 3),
 ('libre', 3),
 ('perseguido', 3),
 ('riunidos', 3),
 ('gran', 3)]

---

## Named Entity Recognition Tagging y aplicaciones

In [25]:
sent_0 = nlp('Joe Biden visitó al presidente de Francia hoy.')
for token in sent_0:
    print(token.text, token.ent_type_)

Joe PER
Biden PER
visitó 
al 
presidente 
de 
Francia LOC
hoy 
. 


Cuando spaCy procesa el documento, las named entitites se guardan en la propiedad `ents` de la clase `Doc` y a nivel de token se guardan en la propiedad `ent_type_` Podemos observar que detectó a `Joe Biden` como una persona y a `Francia` como una ubicación.

In [26]:
sent_1 = nlp("""Emmanuel Jean-Michel Frédéric Macron es un político francés que sirve
                en funciones como presidente de Francia y como co principe de oficio
                de Andorra desde el 14 de mayo de 2017.""")
for token in sent_1:
    print(token.text, token.ent_type_)

Emmanuel PER
Jean-Michel PER
Frédéric PER
Macron PER
es 
un 
político 
francés 
que 
sirve 

                 
en 
funciones 
como 
presidente 
de 
Francia LOC
y 
como 
co 
principe 
de 
oficio 

                 
de 
Andorra LOC
desde 
el 
14 
de 
mayo 
de 
2017 
. 


Aun con un acento francés pudo capturar la entidad `Frédéric`

In [27]:
sent_2 = nlp("""Estudió filosofía en Université Paris Nanterre, completó un Master
                en Asuntos Publicos en Sciences Po y se graduó en la École
                nationale d'administration (ÉNA) en 2004.""")
for token in sent_2:
    print(token.text, token.ent_type_)

Estudió 
filosofía 
en 
Université ORG
Paris ORG
Nanterre ORG
, 
completó 
un 
Master LOC

                 
en 
Asuntos LOC
Publicos LOC
en 
Sciences PER
Po PER
y 
se 
graduó 
en 
la 
École MISC

                 MISC
nationale MISC
d'administration MISC
( 
ÉNA 
) 
en 
2004 
. 


En este texto tenemos palabras en tres idiomas, español, inglés y francés

In [28]:
sent_3 = nlp("""Trabajó en el Inspectorate General of Finances y más tarde se
                convirtió en banquero de inversiones en Rothschild & Cie Banque.""")
for token in sent_3:
    print(token.text, token.ent_type_)

Trabajó PER
en 
el 
Inspectorate MISC
General MISC
of MISC
Finances MISC
y 
más 
tarde 
se 

                 
convirtió 
en 
banquero 
de 
inversiones 
en 
Rothschild LOC
& LOC
Cie LOC
Banque LOC
. 


En este caso vemos que `Trabajó` fue interpretado como persona y `Rothschild & Cie Banque` como un lugar

Podemos visualizar las entidades encontradas con el modulo displaCy

In [29]:
doc = nlp("""Emmanuel Jean-Michel Frédéric Macron es un político francés que sirve en
             funciones como presidente de Francia y como co principe de oficio de
             Andorra desde el 14 de mayo de 2017.
             Estudió filosofía en Université Paris Nanterre, completó un Master en
             Asuntos Publicos en Sciences Po y se graduó en la École nationale
             d'administration (ÉNA) en 2004.
             Trabajó en el Inspectorate General of Finances y más tarde se convirtió
             en banquero de inversiones en Rothschild & Cie Banque.""")        

In [30]:
spacy.displacy.serve(doc, style='ent')


Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


---

## Parseo de dependencias con spaCy

In [31]:
sent_0 = nlp('Myriam vió a Clemente con un telescopio.')
for chunk in sent_0.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_, chunk.root.head.text)

Myriam Myriam nsubj vió
Clemente Clemente obl vió
un telescopio telescopio obl vió


Podemos ver aqui que `vió` es la accion principal, `Myriam` el sujeto y `Clemente` el objeto

In [32]:
spacy.displacy.serve(sent_0, style='dep')


Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [33]:
sent_1 = nlp("""Los vehículos autónomos trasladan la responsabilidad del seguro hacia
                los fabricantes.""")
for chunk in sent_1.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_, chunk.root.head.text)

Los vehículos vehículos nsubj trasladan
la responsabilidad responsabilidad obj trasladan
seguro seguro nmod responsabilidad
los fabricantes fabricantes obj trasladan


In [34]:
spacy.displacy.serve(sent_1, style='dep')


Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


Aqui se pueden ver las tres frases sustantivas `los autos autonomos`, `la responsabilidad` y `seguro`

In [35]:
sent_2 = nlp('Le disparé al elefante con mis pijamas.')
for chunk in sent_2.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_, chunk.root.head.text)

Le Le obj disparé
elefante elefante obj disparé
mis pijamas pijamas obl disparé


In [36]:
spacy.displacy.serve(sent_2, style='dep')


Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


## Ejemplo práctico

Continuando con el ejemplo anterior del Martin Fierro, podemos buscar cuales son los adjetivos que más describe a el `gaucho`, el personaje principal del Martin Fierro

In [37]:
adjectives = []
for sent in mf.sents:
    for word in sent:
        if 'gaucho' in word.text:
            for child in word.children:
                if child.pos_ == 'ADJ':
                    adjectives.append(child.text.strip())
                    
Counter(adjectives).most_common(10)

[('desgraciao', 3),
 ('pobre', 2),
 ('malo', 2),
 ('perseguido', 1),
 ('alvertido', 1),
 ('inteligente', 1),
 ('infeliz', 1),
 ('dormido', 1),
 ('rotoso', 1),
 ('barajo', 1)]

---

## Transformaciones vectoriales con Gensim

Analicemos las primeras 4 estrofas del Martin Fierro

In [38]:
documents = [
'Aquí me pongo a cantar al compás de la vigüela, que el hombre que lo desvela una pena estraordinaria como la ave solitaria con el cantar se consuela.',
'Pido a los Santos del Cielo que ayuden mi pensamiento; Les pido en este momento que voy a cantar mi historia me refresquen la memoria y aclaren mi entendimiento.',
'Vengan Santos milagrosos, vengan todos en mi ayuda, que la lengua se me añuda y se me turba la vista; Pido a Dios que me asista en una ocasión tan ruda.',
'Yo he visto muchos cantores, con famas bien obtenidas, y que después de adquiridas no las quieren sustentar parece que sin largar se cansaron en partidas'
]

Las preprocesamos con spaCy

In [39]:
texts = []
for document in documents:
    text = []
    doc = nlp(document)
    for w in doc:
        if not w.is_stop and not w.is_punct and not w.like_num:
            text.append(w.lemma_)
    texts.append(text)
print(texts)        

[['poner', 'a', 'cantar', 'compás', 'vigüela', 'hombre', 'desvelar', 'pena', 'estraordinario', 'ave', 'solitario', 'cantar', 'consolar'], ['pedir', 'a', 'Santos', 'Cielo', 'ayudar', 'pensamiento', 'pedir', 'a', 'cantar', 'historia', 'refrescar', 'memoria', 'y', 'aclaren', 'entendimiento'], ['Vengan', 'Santos', 'milagroso', 'vengar', 'ayuda', 'lengua', 'añudar', 'y', 'turbar', 'vista', 'pedir', 'a', 'Dios', 'asistir', 'ocasión', 'rudo'], ['ver', 'cantor', 'fama', 'obtenido', 'y', 'adquirida', 'querer', 'sustentar', 'largar', 'cansar', 'partida']]


Construyamos nuestro Bag-of-words

In [40]:
dictionary = corpora.Dictionary(texts)
print(dictionary.token2id)

{'a': 0, 'ave': 1, 'cantar': 2, 'compás': 3, 'consolar': 4, 'desvelar': 5, 'estraordinario': 6, 'hombre': 7, 'pena': 8, 'poner': 9, 'solitario': 10, 'vigüela': 11, 'Cielo': 12, 'Santos': 13, 'aclaren': 14, 'ayudar': 15, 'entendimiento': 16, 'historia': 17, 'memoria': 18, 'pedir': 19, 'pensamiento': 20, 'refrescar': 21, 'y': 22, 'Dios': 23, 'Vengan': 24, 'asistir': 25, 'ayuda': 26, 'añudar': 27, 'lengua': 28, 'milagroso': 29, 'ocasión': 30, 'rudo': 31, 'turbar': 32, 'vengar': 33, 'vista': 34, 'adquirida': 35, 'cansar': 36, 'cantor': 37, 'fama': 38, 'largar': 39, 'obtenido': 40, 'partida': 41, 'querer': 42, 'sustentar': 43, 'ver': 44}


Hay 45 palabras únicas en el corpus, representadas en el diccionario con un indice asignado

In [41]:
len(dictionary)

45

In [42]:
corpus = [dictionary.doc2bow(text) for text in texts]

Al imprimirlo obtenemos una lista de 4 listas (las estrofas), y en cada una de ellas hay una tupla con el índice de la palabra y su frecuencia (word_id, word_count)

In [43]:
print(corpus)

[[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1)], [(0, 2), (2, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 2), (20, 1), (21, 1), (22, 1)], [(0, 1), (13, 1), (19, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1)], [(22, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1)]]


Hay muchos conteos de uno porque es un corpus pequeño.
Podemos guardar esta representacion en disco para recuperarla mas tarde:

In [44]:
corpora.MmCorpus.serialize('models/mf.mm', corpus)

Podemos optar por la representación TF-IDF, con el corpus anterior hacemos:

In [45]:
tfidf = models.TfidfModel(corpus)

Si lo visualizamos podemos obtener estos scores, cuanto más grande, mas importante es en el documento

In [46]:
for document in tfidf[corpus]:
    print(document)

[(0, 0.06244713910698048), (1, 0.3009228766821634), (2, 0.3009228766821634), (3, 0.3009228766821634), (4, 0.3009228766821634), (5, 0.3009228766821634), (6, 0.3009228766821634), (7, 0.3009228766821634), (8, 0.3009228766821634), (9, 0.3009228766821634), (10, 0.3009228766821634), (11, 0.3009228766821634)]
[(0, 0.1331553986738668), (2, 0.16041369623857302), (12, 0.32082739247714603), (13, 0.16041369623857302), (14, 0.32082739247714603), (15, 0.32082739247714603), (16, 0.32082739247714603), (17, 0.32082739247714603), (18, 0.32082739247714603), (19, 0.32082739247714603), (20, 0.32082739247714603), (21, 0.32082739247714603), (22, 0.0665776993369334)]
[(0, 0.05849399300299983), (13, 0.14093664573595682), (19, 0.14093664573595682), (22, 0.05849399300299983), (23, 0.28187329147191365), (24, 0.28187329147191365), (25, 0.28187329147191365), (26, 0.28187329147191365), (27, 0.28187329147191365), (28, 0.28187329147191365), (29, 0.28187329147191365), (30, 0.28187329147191365), (31, 0.28187329147191365

## n-gramas

In [47]:
text = []
doc = nlp(martin_fierro)
for w in doc:
    if not w.is_stop and not w.is_punct and not w.like_num and '\n' not in w.text:
        text.append(w.lemma_.lower())

In [48]:
bigram = models.Phrases([text])

In [49]:
bigram[text][:10]

['gaucho',
 'martín_fierro',
 'josé',
 'hernández',
 'aires',
 'imprenta',
 'pampa',
 'victoria',
 'carta',
 'autor']

In [50]:
bigram_counter = Counter()
for key in bigram.vocab.keys():
    if len(key.split("_")) > 1:
        bigram_counter[key] += bigram.vocab[key]

In [51]:
for key, counts in bigram_counter.most_common(20):
    print(key, counts)

y_a 17
ir_a 10
a_cantar 8
y_pa 7
empezar_a 7
y_venir 7
martín_fierro 6
y_dendir 6
a_andar 6
y_ansí 6
y_salir 6
gaucho_y 5
y_gaucho 5
y_querer 5
y_ver 5
y_andar 5
a_frontera 5
y_hacer 5
y_decir 5
a_buscar 5


## Topic Modeling

### Latent Dirichlet Allocation

In [52]:
lee_newspaper = open('corpus/lee_background.cor').read()

In [54]:
# !python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")

In [55]:
doc = nlp(lee_newspaper)

In [56]:
doc[:100]

Hundreds of people have been forced to vacate their homes in the Southern Highlands of New South Wales as strong winds today pushed a huge bushfire towards the town of Hill Top. A new blaze near Goulburn, south-west of Sydney, has forced the closure of the Hume Highway. At about 4:00pm AEDT, a marked deterioration in the weather as a storm cell moved east across the Blue Mountains forced authorities to make a decision to evacuate people from homes in outlying streets at Hill Top in the New South Wales southern highlands.

In [57]:
# we add some words to the stop word list
texts, article, skl_texts = [], [], []
for w in doc:
    # if it's not a stop word or punctuation mark, add it to our article!
    if w.text != '\n' and not w.is_stop and not w.is_punct and not w.like_num:
        # we add the lematized version of the word
        article.append(w.lemma_)
    # if it's a new line, it means we're onto our next document
    if w.text == '\n':
        skl_texts.append(' '.join(article))
        texts.append(article)
        article = []

In [58]:
bigram = models.Phrases(texts)

In [59]:
texts = [bigram[line] for line in texts]

In [60]:
texts[1][0:10]

['indian',
 'security_force',
 'shoot_dead',
 'suspect',
 'militant',
 'night',
 'long',
 'encounter',
 'southern',
 'Kashmir']

In [61]:
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [62]:
corpus[1][0:10]

[(71, 1),
 (83, 1),
 (92, 1),
 (94, 1),
 (95, 1),
 (110, 1),
 (111, 1),
 (112, 1),
 (113, 4),
 (114, 1)]

In [63]:
ldamodel = LdaModel(corpus=corpus, num_topics=10, id2word=dictionary)

In [64]:
ldamodel.show_topics()

[(0,
  '0.038*"say" + 0.006*"Mr" + 0.005*"people" + 0.004*"report" + 0.004*"Australian" + 0.004*"come" + 0.004*"Australia" + 0.004*"Government" + 0.003*"Taliban" + 0.003*"year"'),
 (1,
  '0.024*"say" + 0.005*"year" + 0.004*"Australia" + 0.004*"force" + 0.004*"Mr" + 0.004*"day" + 0.004*"new" + 0.003*"take" + 0.003*"good" + 0.003*"Afghanistan"'),
 (2,
  '0.034*"say" + 0.006*"Mr" + 0.005*"force" + 0.004*"attack" + 0.004*"palestinian" + 0.004*"Mr_Arafat" + 0.004*"people" + 0.004*"israeli" + 0.004*"Afghanistan" + 0.003*"group"'),
 (3,
  '0.023*"say" + 0.006*"day" + 0.005*"Australia" + 0.005*"Mr" + 0.004*"go" + 0.004*"today" + 0.003*"know" + 0.003*"people" + 0.003*"Test" + 0.003*"place"'),
 (4,
  '0.019*"say" + 0.006*"Australia" + 0.005*"test" + 0.005*"day" + 0.005*"fire" + 0.005*"South_Africa" + 0.005*"wicket" + 0.004*"good" + 0.003*"run" + 0.003*"match"'),
 (5,
  '0.020*"say" + 0.011*"Mr" + 0.004*"israeli" + 0.004*"Australia" + 0.004*"force" + 0.004*"people" + 0.004*"year" + 0.003*"palesti

Es una lista de tuplas con el primer valor indentificando el tema y a continuacion la distribucion de probabilidad de los tokens mayoritarios

In [65]:
pyLDAvis.enable_notebook()

In [66]:
lda_viz = gensimvis.prepare(ldamodel, corpus, dictionary)
lda_viz

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.010791 -0.034219       1        1  26.613268
0     -0.015541  0.000296       2        1  18.954718
8     -0.034042  0.017552       3        1   9.773784
5     -0.023995 -0.002734       4        1   9.750906
7     -0.015988  0.075985       5        1   7.447208
4      0.086460  0.001148       6        1   6.638744
1      0.026858 -0.020469       7        1   6.235847
3      0.039693  0.011706       8        1   5.844563
6     -0.028153 -0.044082       9        1   4.629119
9     -0.024501 -0.005183      10        1   4.111842, topic_info=             Term        Freq       Total Category  logprob  loglift
90            say  826.000000  826.000000  Default  30.0000  30.0000
310     Australia  108.000000  108.000000  Default  29.0000  29.0000
1015          man   64.000000   64.000000  Default  28.0000  28.0000
2341      company   58.000000   58.000000  Default  27.0000  27.0000
219           day   97.000000   97.000000  Default  26.0000  26.0000
...           ...         ...         ...      ...      ...      ...
843   palestinian    2.987685   69.316382  Topic10  -6.0174   0.0471
225           end    2.436853   43.671121  Topic10  -6.2212   0.3053
129        attack    2.661238   79.807213  Topic10  -6.1331  -0.2095
882        report    2.455044   70.806770  Topic10  -6.2138  -0.1705
219           day    2.438089   97.918497  Topic10  -6.2207  -0.5016

[796 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
635       1  0.320337           
635       2  0.149491           
635       3  0.042712           
635       4  0.149491           
635       5  0.064067           
...     ...       ...        ...
178       6  0.094693  yesterday
178       7  0.071020  yesterday
178       8  0.047347  yesterday
178       9  0.023673  yesterday
178      10  0.023673  yesterday

[2118 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 9, 6, 8, 5, 2, 4, 7, 10])

Podemos ver que palabras como `say`, `Mr`, `said`, `says` y `saying` aparecen en casi todos los topicos, asi que podriamos agregarlas a las stopwords y hacer una nueva iteracion